# Ruokavalion optimointi - mitä ruokia ja kuinka paljon pitää syödä, jotta päivittäiset saantisuositukset täyttyvät?

##### optimoinnin tulos lista kertoimia (satoina grammoina), joiden mukainen määrä kutakin elintarviketta tulee syödä

ravintosisältötiedot peräisin Finelin versiosta 20

rajoituksia:
* kutakin ravintoainetta oltava (vähintään?) suositusten mukainen määrä
* kaikki kertoimet 0 - 900 (?)
* kertoimien summa alle 5000
* mahdollisimman pieni määrä nollasta poikkeavia kertoimia
* ruokien yhteenlaskettu energiamäärä 2000 kcal (* 4,184) - tai mahdollisimman pieni?

Työnkulku:
* luetaan 5 csv-tiedostoa: component_value_utf-8.csv, eufdname_FI_utf-8.csv, food_utf-8.csv, saantisuositus_2014.csv ja component_utf-8.csv -> read_files()
* poistetaan food-taulusta lisäravinteet, laihdutusvalmisteet, äidinmaidonkorvikkeet yms. -> filter_food_class()
* poistetaaan food-taulusta myös joitain yksittäisiä tuotteita (mm. makeutusaineet, liivate) nimen perusteella
* poistetaan component_value-taulusta rivit, jotka liittyvät edellisessä vaiheessa poistettuihin valmisteisiin -> filter_components_by_food()
* tehdään component_value-taululle pivot: ennen jokaisella ruoka + ravinne -parilla oli oma rivi, nyt kunkin ruuan tiedot kerätään yhdelle riville ja ravinteet ovat omissa sarakkeissaan
* luodaan scipy.optimize.linprog()-funktion parametrit A, b, c, bounds
  - A-matriisi: listojen lista, jossa kukin alilista on yhden ravinteen määrät eri ruuissa -> create_matrix_a_eq()
  - b-vektori, joka sisältää saantisuositukset
  - c-vektori, joka sisältää eri ruokien energiamäärät -> get_single_component_values()
  - bounds: yhden ruoan määrän yläraja (tällä hetkellä 8 eli 800 g)
  
#TODO: miten saa painettua minimaaliset kertoimet nollaan?

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

In [32]:
# pois laskuista jätettävät elintarviketyypit (FUCLASS):

# omitted_food_types:
# lastenruoat, äidinmaidonkorvikkeet, ateriankorvikkeet, lisäravinteet, 
# keinotekoisesti makeutetut virvoitusjuomat, urheilujuomat ja kivennäisvedet
# (kivennäisvedet lähinnä siksi, että koska niissä ei ole energiaa, algoritmilla
# ei ole mitään tarvetta rajoittaa niiden määrää, jolloin huomattava osa
# kivennäisistä tulee niistä)

omitted_food_types = ('BABYFTOT', 
'BABMEATD',
'BABFISHD',
'BABMILPO',
'BABWATPO',
'BABFRUB',
'BABVEGE',
'BABMIFRU',
'BABOTHER',
'MMILK',
'INFMILK',
'CASMILK',
'PREMILK',
'SOYMILK',
'WHEYMILK',
'AMINMILK',
'SPECTOT',
'SPECSUPP',
'MEALREP',
'SPORTFOO',
'SPECFOOD',
'DRINKART',
'DRSPORT',
'DRWATER')

stricter_food_types = ('BABYFTOT', 
'BABMEATD',
'BABFISHD',
'BABMILPO',
'BABWATPO',
'BABFRUB',
'BABVEGE',
'BABMIFRU',
'BABOTHER',
'MMILK',
'INFMILK',
'CASMILK',
'PREMILK',
'SOYMILK',
'WHEYMILK',
'AMINMILK',
'SPECTOT',
'SPECSUPP',
'MEALREP',
'SPORTFOO',
'SPECFOOD',
'DRINKART',
'DRSPORT',
'DRWATER',
                      'DRINKSO',
                      'INGRMISC',
                      'SPICES',
                      'SPISAUCE')

In [3]:
def read_files(path:str) -> tuple:
    """This function reads the following csv files and returns 
    a tuple of pandas data structures: 
    component_value_utf-8.csv
    eufdname_FI_utf-8.csv
    food_utf-8.csv
    saantisuositus_2014.csv

    The function also removes data for various supplements,
    since the target is to look at real foods.

    Args:
        path (str): absolute path to csv files

    Returns:
        tuple: Returns a tuple of pandas data structures with the 
        data from the csv files. (component_value, eufdname, food,
        saantisuositus)
    """

    component_value = pd.read_csv(path + "component_value_utf-8.csv", sep=";", keep_default_na=False)
    eufdname = pd.read_csv(path + "eufdname_FI_utf-8.csv", sep=";")
    food = pd.read_csv(path + "food_utf-8.csv", sep=";")
    saantisuositus = pd.read_csv(path + "saantisuositus_2014.csv", sep=";", header=None, names=["EUFDNAME", "name", "mnuori", "maikuinen", "mkeski", "miäkäs", "mvanha", "npieni","nnuori", "naikuinen", "nkeski", "niäkäs", "nvanha"])
    component = pd.read_csv(path + "component_utf-8.csv", sep=";", dtype={'EUFDNAME':str}, keep_default_na=False)

    return component_value, eufdname, food, saantisuositus, component

In [4]:
component_value, eufdname, food, saantisuositus, component = read_files('/home/pomo/Asiakirjat/Kurssit/Taitotalo_Python-ohjelmoija/python/portfolio/fineli_20/')
print(component_value.head())
print(eufdname.head())
print(food.head())
print(saantisuositus.head())
print(component.head())
print(component_value['EUFDNAME'][45:50])
print(component_value['BESTLOC'][45:50])

/home/pomo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


   FOODID EUFDNAME  BESTLOC ACQTYPE METHTYPE METHIND
0       1    ENERC  1698.30       S        S  MI0107
1       1      FAT        0       F        E  MIR003
2       1   CHOAVL   99.900       S        S  MI0181
3       1   CHOCDF   99.880       S        S  MI0131
4       1     PROT        0       L        P  MIR003
  THSCODE                              DESCRIPT LANG
0   ENERC               energia, laskennallinen   FI
1  XENERC  energia, laskennallinen ilman kuitua   FI
2     FAT                                 rasva   FI
3  CHOAVL               hiilihydraatti imeytyvä   FI
4  CHOCDF             hiilihydraatti erotuksena   FI
   FOODID                  FOODNAME FOODTYPE PROCESS  EDPORT   IGCLASS  \
0       1                    SOKERI     FOOD     IND     100  SUGARSYR   
1       2  FRUKTOOSI, HEDELMÄSOKERI     FOOD     IND     100  SUGARSYR   
2       3                  SIIRAPPI     FOOD     IND     100  SUGARSYR   
3       4                    HUNAJA     FOOD     RAW     100  SUGARS

In [5]:
# poistetaan rivit, joissa ei ole BESTLOC-arvoa

component_value = component_value[component_value.BESTLOC != ""]

# muutetaan object-tyyppiset luvut liukuluvuiksi
component_value['BESTLOC'] = component_value['BESTLOC'].astype(float)
print(component_value['BESTLOC'][45:50])

45    0.01
46    0.01
47    0.10
48    0.25
49    0.00
Name: BESTLOC, dtype: float64


In [6]:
# get the full list of components (=nutrients)
component_list = sorted(component['EUFDNAME'].tolist())
component_list

['ALC',
 'ASH',
 'CA',
 'CAROTENS',
 'CARTB',
 'CHOAVL',
 'CHOCDF',
 'CHOLE',
 'CR',
 'CU',
 'ENERC',
 'F16D0T',
 'F18D1T',
 'F18D2CN6',
 'F18D3N3',
 'F20D4N6',
 'F20D5N3',
 'F22D6N3',
 'FACIDCTG',
 'FAFRE',
 'FAMCIS',
 'FAPU',
 'FAPUN3',
 'FAPUN6',
 'FAS18',
 'FASAT',
 'FAT',
 'FATRN',
 'FD',
 'FE',
 'FIBC',
 'FIBINS',
 'FIBT',
 'FOL',
 'FRUS',
 'GALS',
 'GLUS',
 'ID',
 'K',
 'LACS',
 'MALS',
 'MG',
 'MN',
 'MO',
 'MYRIC',
 'NA',
 'NACL',
 'NIA',
 'NIAEQ',
 'NT',
 'OA',
 'P',
 'PROT',
 'PSACNCS',
 'QUERCE',
 'RETOL',
 'RIBF',
 'SE',
 'STARCH',
 'STERT',
 'SUCS',
 'SUGAR',
 'SUGOH',
 'THIA',
 'TRP',
 'VITA',
 'VITB12',
 'VITC',
 'VITD',
 'VITE',
 'VITK',
 'VITPYRID',
 'WATER',
 'ZN']

In [63]:
def filter_food_class(dataframe:pd.DataFrame, class_to_remove:tuple) -> pd.DataFrame:
    """Take a pandas dataframe with food data and remove
    lines where the FUCLASS is one of those specified in 
    fuclass_to_remove. Returns the cleaned dataframe.

    Args:
        dataframe (pandas_df): A pandas dataframe with food names and food use class (FUCLASS)
        class_to_remove (tuple): A tuple of FUCLASS values to use for filtering out unwanted data from the dataframe

    Returns:
        pandas_df: The pandas dataframe minus the lines with specified food_class values
    """
    for food_class in class_to_remove:
        dataframe = dataframe[dataframe.FUCLASS != food_class]

    return dataframe

In [8]:
# puuttuvien ravintoarvotietojen käsittely

def transpose_component_value(dataframe:pd.DataFrame) -> pd.DataFrame:
    """Takes a pandas dataframe (component_value) where each row represents
    the amount of one nutrient in one food (e.g. calcium in milk). 
    Transposes the dataframe so that there is one row per food and one column per nutrient.
    Fills in any missing component values with zeroes in the appropriate column.
    The ACQTYPE, METHTYPE, and METHIND columns are dropped.

    Args:
        dataframe (pandas_df): A pandas dataframe with information on how much of each nutrient foods contain

    Returns:
        pandas_df: The pandas dataframe modified so that it has one row per food and one column per nutrient.
    """
    df = dataframe.drop(columns=['ACQTYPE','METHTYPE', 'METHIND'])

    new_df = df.pivot_table(values='BESTLOC', index='FOODID', columns='EUFDNAME', fill_value=0)
    
    return new_df
    
df = transpose_component_value(component_value)
df.head()

EUFDNAME,ALC,ASH,CA,CAROTENS,CARTB,CHOAVL,CHOCDF,CHOLE,CR,CU,...,TRP,VITA,VITB12,VITC,VITD,VITE,VITK,VITPYRID,WATER,ZN
FOODID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.02,0.4,0.0,0.0,99.9,99.88,0.0,1.0,0.010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.01
2,0.0,0.00,0.1,0.0,0.0,99.8,99.70,0.0,1.0,0.010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.3,0.01
3,0.0,1.10,74.0,0.0,0.0,74.1,79.30,0.0,5.0,0.240,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,19.5,0.13
4,0.0,0.18,5.0,0.0,0.0,80.8,81.32,0.0,5.0,0.030,...,0.0,0.0,0.0,4.0,0.0,0.0,0.02,0.0,18.0,0.10
5,0.0,0.90,7.0,0.0,0.0,87.0,88.60,0.0,26.0,0.175,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,10.0,0.29


In [9]:
def filter_components_by_food(component_value:pd.DataFrame, food:pd.DataFrame) -> pd.DataFrame:
    """Takes two pandas dataframes: component_value and food. 
    Removes unnecessary rows from component_value 
    to only include rows that have FOODID values found in food.
    If the food dataframe was previously filtered with filter_food_class(),
    we don't want to drag along rows related to foods that are no longer in the 
    food dataframe.

    Args:
        component_value (pandas_df): A dataframe with information on how much of each nutrient foods contain
        food (pandas_df): A dataframe with foods

    Returns:
        pandas_df: A filtered version of component_value
    """
    food_ids = food['FOODID'].tolist()
    new_component_value = component_value[component_value.FOODID.isin(food_ids)]
    
    return new_component_value

In [12]:
def get_single_component_values(component_value:pd.DataFrame, component="ENERC") -> list:
    """Takes a pandas dataframe with the nutrient compositions of foods and the EUFDNAME of a single nutrient. 
    Returns the values of a specified component for the foods included.

    Args:
        component_value (pandas_df): A dataframe with information on how much of each nutrient foods contain
        component (string): the EUFDNAME of a nutrient

    Returns:
        list: A list of the values of the specified component (default: ENERC) for the foods
    """
    specified_only = component_value[component_value.EUFDNAME == component]
    
    return specified_only['BESTLOC'].tolist()

In [13]:
# uusi versio funktiosta uudella, pivotoidulla component_value-taululla
def create_matrix_a_eq(component_value:pd.DataFrame, nutrient_list:list) -> list:
    """Takes a pandas dataframe with the nutrient compositions of foods and
    a list of EUFDNAMEs of nutrients. 
    Returns the A_eq matrix (list of lists) for use in the scipy.optimize.linprog function.

    Args:
        component_value (pandas_df): A dataframe with information on how much of each nutrient foods contain
        nutrient_list (list): A list of EUFDNAMEs

    Returns:
        list of lists: The A_eq matrix where each row is the amounts of one nutrient in the various foods.
    """
    A = []
    for nutrient in nutrient_list:
        A.append(component_value[nutrient].tolist())
        
    return A

In [14]:
def create_rda_list(rda:pd.DataFrame, nutrient_list:list, target_group:str="nkeski") -> list:
    """Takes a pandas dataframe with the recommended daily allowances for various nutrients
    and a list of EUFDNAMEs of nutrients. 
    Returns the A_eq matrix (list of lists) for use in the scipy.optimize.linprog function.

    Args:
        rda (pandas_df): A dataframe with rda values for nutrients
        nutrient_list (list): A list of EUFDNAMEs
        target_group (str): The age & sex combination of the target group (default: middle-aged women)

    Returns:
        list: The b constraint vector where each value is the rda of a nutrient.
    """
    specified_only = rda[rda.EUFDNAME.isin(nutrient_list)]
    
    return specified_only[target_group].tolist()

In [15]:
print(len(food.index))
testi = food.loc[food['FOODID']<24990]
print(len(testi.index))
testi.head()


4238
1378


,FOODID,FOODNAME,FOODTYPE,PROCESS,EDPORT,IGCLASS,IGCLASSP,FUCLASS,FUCLASSP
0,1,SOKERI,FOOD,IND,100,SUGARSYR,SUGARTOT,SUGADD,SUGARTOT
1,2,"FRUKTOOSI, HEDELMÄSOKERI",FOOD,IND,100,SUGARSYR,SUGARTOT,SUGADD,SUGARTOT
2,3,SIIRAPPI,FOOD,IND,100,SUGARSYR,SUGARTOT,SUGADD,SUGARTOT
3,4,HUNAJA,FOOD,RAW,100,SUGARSYR,SUGARTOT,SUGADD,SUGARTOT
4,5,"MAKEINEN, KARAMELLI",FOOD,IND,100,SWEET,SUGARTOT,SWEET,SUGARTOT


In [64]:
#filter food
filtered_food = filter_food_class(food, stricter_food_types)
print(len(filtered_food.index))

# lisäksi poistetaan joitain muita sekalaisia
filtered_food = filtered_food[~filtered_food["FOODNAME"].str.contains("MAKEUTUSAINE")]
filtered_food = filtered_food[~filtered_food["FOODNAME"].str.contains("KARPPISOKERI")]
filtered_food = filtered_food[~filtered_food["FOODNAME"].str.contains("LIIVATE")]
print(len(filtered_food.index))


3997
3993


In [34]:
#filter component_value by foodid
print(len(component_value))
filtered_comp_val = filter_components_by_food(component_value, filtered_food)
print(len(filtered_comp_val))
filtered_comp_val.tail()

307819
291362


,FOODID,EUFDNAME,BESTLOC,ACQTYPE,METHTYPE,METHIND
307829,35887,CHOLE,323.57,S,R,MI0002
307830,35887,STERT,11.43,S,R,MI0002
307831,35887,TRP,486.20,S,R,MI0002
307832,35887,MYRIC,0.00,S,R,MI0002
307833,35887,QUERCE,0.00,S,R,MI0002


In [35]:
# pivot component_value
pivoted_component_value = transpose_component_value(filtered_comp_val)
pivoted_component_value.head()

EUFDNAME,ALC,ASH,CA,CAROTENS,CARTB,CHOAVL,CHOCDF,CHOLE,CR,CU,...,TRP,VITA,VITB12,VITC,VITD,VITE,VITK,VITPYRID,WATER,ZN
FOODID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.02,0.4,0.0,0.0,99.9,99.88,0.0,1.0,0.010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.01
2,0.0,0.00,0.1,0.0,0.0,99.8,99.70,0.0,1.0,0.010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.3,0.01
3,0.0,1.10,74.0,0.0,0.0,74.1,79.30,0.0,5.0,0.240,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,19.5,0.13
4,0.0,0.18,5.0,0.0,0.0,80.8,81.32,0.0,5.0,0.030,...,0.0,0.0,0.0,4.0,0.0,0.0,0.02,0.0,18.0,0.10
5,0.0,0.90,7.0,0.0,0.0,87.0,88.60,0.0,26.0,0.175,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,10.0,0.29


In [36]:
c = pivoted_component_value['ENERC'].tolist()
print(len(c))
print(c[:10])

3992
[1698.3, 1696.6, 1261.4, 1382.1, 1487.5, 2270.15, 1358.84, 1375.56, 1294.8, 2033.21]


In [37]:
# koko saantisuosituslistalla pois lukien ENERC

rda_minus_enerc = saantisuositus['EUFDNAME'].tolist()[:-1] # olettaa, että ENERC on viimeisenä

A = create_matrix_a_eq(pivoted_component_value, rda_minus_enerc)
b = create_rda_list(saantisuositus, nutrient_list=rda_minus_enerc, target_group="nkeski")
res = linprog(c, A_eq=A, b_eq=b, bounds=(0, 8))
res

     con: array([-4.51028654e-07, -3.35446870e-09, -4.14328838e-09, -3.58716390e-10,
       -4.57929250e-10, -9.71392922e-09, -5.14110976e-10, -8.15236945e-08,
       -2.69851830e-09, -1.87007032e-08, -2.09666382e-07, -4.00914587e-07,
       -7.66841822e-07, -9.02880970e-08, -3.93385768e-09, -3.43335138e-09,
       -2.84752455e-09, -4.20875352e-07, -2.91436706e-08, -6.81676227e-09,
       -2.08187245e-08, -2.46612473e-08, -2.19480080e-06])
     fun: 2479.7282468785447
 message: 'Optimization terminated successfully.'
     nit: 52
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([2.59994069e-12, 2.60059563e-12, 3.88267159e-12, ...,
       3.70551228e-13, 2.68434679e-09, 7.27737816e-14])

In [56]:
foodid_list = pivoted_component_value.index.tolist()
foodnames = pd.merge(pivoted_component_value, filtered_food, how='inner', on='FOODID')[['FOODID', 'FOODNAME']]
foodname_list = foodnames['FOODNAME'].tolist()

# määrä satoina grammoina, foodid, foodname: järjestetty määrän mukaan laskevaan järjestykseen
amount_id_name = sorted(zip(res.x, foodid_list, foodname_list), reverse=True)
amount_id_name[:30]

[(7.999999999690903, 923, 'TEE, YRTTITEE'),
 (2.692436175282419, 914, 'KALJA, KOTIOLUT'),
 (1.2408406784170072, 34691, 'KANANMUNAN VALKUAINEN, KEITETTY'),
 (1.1959193061645748, 360, 'MANGOLDI'),
 (0.9864464318905342, 33583, 'LATVA-ARTISOKKA, KEITETTY'),
 (0.9141366601335386,
  33116,
  'JUUSTO, SALAATTIJUUSTO, RASVAA 2-5 %, LEHMÄNMAITO, SUOLALIEMESSÄ'),
 (0.6986143843409193, 459, 'MEHU, MEHUJUOMA, PUOLUKKA, LAIMENTAMATON'),
 (0.6172791461356802, 4409, 'MEHUHYYTELÖ'),
 (0.4985772158256635, 34237, 'VUONANKAALI'),
 (0.4832018951596205, 29684, 'TURSKA, KEITETTY, SUOLATON'),
 (0.45704480066710473, 11689, 'PARSA, RAAKA, PUHDISTETTU'),
 (0.4394858143204485,
  33885,
  'RAHKA, MAUSTETTU, PROFEEL PROTEIINIRAHKA, LAKTOOSITON'),
 (0.41156958391787873, 34159, 'RUKOLA, RUCOLA, SINAPPIKAALI'),
 (0.283279355640211, 34537, 'SUPPILOVAHVERO, KOKOON KEITETTY'),
 (0.21296878091609014, 863, 'OSTERI'),
 (0.20560248681216506, 409, 'APRIKOOSI, KUIVATTU'),
 (0.19173133509615684, 33925, 'QUORN, FILEET, KUUTIOT,